In [376]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score

In [377]:
train_values = pd.read_csv('../Archivos de datos/train_values.csv')
train_labels = pd.read_csv('../Archivos de datos/train_labels.csv')
test_values  = pd.read_csv('../Archivos de datos/test_values.csv')

In [378]:
train_values.head(2)

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,0


In [379]:
train = train_values.merge(train_labels)
label=train.damage_grade

#'has_secondary_use',
#             'has_secondary_use_agriculture','has_secondary_use_hotel','has_secondary_use_rental',
#             'has_secondary_use_institution','has_secondary_use_school','has_secondary_use_industry',
#             'has_secondary_use_health_post','has_secondary_use_gov_office',
#             'has_secondary_use_use_police','has_secondary_use_other',
#,'has_superstructure_adobe_mud',
#             'has_superstructure_mud_mortar_stone','has_superstructure_stone_flag',
#             'has_superstructure_cement_mortar_stone','has_superstructure_mud_mortar_brick',
#             'has_superstructure_cement_mortar_brick','has_superstructure_timber',
#             'has_superstructure_bamboo','has_superstructure_rc_non_engineered',
#             'has_superstructure_rc_engineered','has_superstructure_other',
train=train[['geo_level_1_id','geo_level_2_id','geo_level_3_id',
             
             'land_surface_condition','foundation_type','ground_floor_type','age','plan_configuration',
             'height_percentage','roof_type','other_floor_type','position']]

In [380]:
p=pd.get_dummies(train.position)
p_columns = p.columns
p_columns=np.array(p_columns)

In [381]:
def position(p_columns):
    y=[]
    for i in p_columns:
        y.append('position_' + str(i))
    return y

In [382]:
p.columns=surface(p_columns)
train[p.columns]=p
train=train.drop(columns = ['position'])

In [383]:
oft=pd.get_dummies(train.other_floor_type)
oft_columns = oft.columns
oft_columns=np.array(oft_columns)

In [384]:
def other_floor_type(oft_columns):
    y=[]
    for i in rt_columns:
        y.append('other_floor_type_' + str(i))
    return y

In [385]:
oft.columns=surface(oft_columns)
train[oft.columns]=oft
train=train.drop(columns = ['other_floor_type'])

In [386]:
rt=pd.get_dummies(train.roof_type)
rt_columns = rt.columns
rt_columns=np.array(rt_columns)

In [387]:
def roof_type(rt_columns):
    y=[]
    for i in rt_columns:
        y.append('roof_type_' + str(i))
    return y

In [388]:
rt.columns=surface(rt_columns)
train[rt.columns]=rt
train=train.drop(columns = ['roof_type'])

In [389]:
pc=pd.get_dummies(train.plan_configuration)
pc_columns = pc.columns
pc_columns=np.array(pc_columns)

In [390]:
def plan_configuration(pc_columns):
    y=[]
    for i in pc_columns:
        y.append('plan_configuration_' + str(i))
    return y

In [391]:
pc.columns=surface(pc_columns)
train[pc.columns]=pc
train=train.drop(columns = ['plan_configuration'])

In [392]:
x=pd.get_dummies(train.land_surface_condition)
x_columns = x.columns
x_columns=np.array(x_columns)

In [393]:
def surface(x_columns):
    y=[]
    for i in x_columns:
        y.append('land_surface_condition_' + str(i))
    return y

In [394]:
x.columns=surface(x_columns)
train[x.columns]=x
train=train.drop(columns = ['land_surface_condition'])

In [395]:
z=pd.get_dummies(train.foundation_type)
z_columns = z.columns
z_columns=np.array(z_columns)

In [396]:
def foundation(x_columns):
    y=[]
    for i in x_columns:
        y.append('foundation_type_' + str(i))
    return y

In [397]:
z.columns=foundation(z_columns)
train[z.columns]=z
train=train.drop(columns = ['foundation_type'])

In [398]:
g=pd.get_dummies(train.geo_level_1_id)
g_columns = g.columns
g_columns=np.array(g_columns)

In [399]:
def geo_level_1(g_columns):
    y=[]
    for i in g_columns:
        y.append('geo_level_1_id_' + str(i))
    return y

In [400]:
g.columns=foundation(g_columns)
train[g.columns]=g
train=train.drop(columns = ['geo_level_1_id'])

In [401]:
gf=pd.get_dummies(train.ground_floor_type)
gf_columns = gf.columns
gf_columns=np.array(gf_columns)

In [402]:
def ground_floor_type(gf_columns):
    y=[]
    for i in gf_columns:
        y.append('ground_floor_type_' + str(i))
    return y

In [403]:
 ground_floor_type(gf_columns)

['ground_floor_type_f',
 'ground_floor_type_m',
 'ground_floor_type_v',
 'ground_floor_type_x',
 'ground_floor_type_z']

In [404]:
gf.columns=ground_floor_type(gf_columns)
train[gf.columns]=gf
train=train.drop(columns = ['ground_floor_type'])

In [405]:
#boosting_type='gbdt', objective='binary', num_leaves=200,
#                                learning_rate=0.1, n_estimators=500, max_depth=100,
#                                bagging_fraction=0.9, feature_fraction=0.9, reg_lambda=0.2
lgb_clf = lgb.LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=200,
                                learning_rate=0.1, n_estimators=500, max_depth=100,
                                bagging_fraction=0.9, feature_fraction=0.9, reg_lambda=0.2)

In [406]:
X_train, X_test, y_train, y_test = train_test_split(train, label, test_size = 0.3, random_state = 0)

In [407]:
lgb_clf.fit(X_train, y_train)

[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9


LGBMClassifier(bagging_fraction=0.9, feature_fraction=0.9, max_depth=100,
               n_estimators=500, num_leaves=200, objective='binary',
               reg_lambda=0.2)

In [408]:
x_predict=lgb_clf.predict(X_test)

In [409]:
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score

In [410]:
precision_score(x_predict,y_test,average=None).mean()

0.6591200547279633

In [411]:
test_values.head(2)

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,...,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,...,1,0,0,0,0,0,0,0,0,0


In [412]:
test_values=test_values[['geo_level_1_id','geo_level_2_id','geo_level_3_id',
                        'land_surface_condition','foundation_type','ground_floor_type',
                         'age','plan_configuration',
             'height_percentage','roof_type','other_floor_type','position']]

In [413]:
p=pd.get_dummies(test_values.position)
p_columns = p.columns
p_columns=np.array(p_columns)
p.columns=surface(p_columns)
test_values[p.columns]=p
test_values=test_values.drop(columns = ['position'])

In [414]:
oft=pd.get_dummies(test_values.other_floor_type)
oft_columns = oft.columns
oft_columns=np.array(oft_columns)
oft.columns=surface(oft_columns)
test_values[oft.columns]=oft
test_values=test_values.drop(columns = ['other_floor_type'])

In [415]:
rt=pd.get_dummies(test_values.roof_type)
rt_columns = rt.columns
rt_columns=np.array(rt_columns)
rt.columns=surface(rt_columns)
test_values[rt.columns]=rt
test_values=test_values.drop(columns = ['roof_type'])

In [416]:
pc=pd.get_dummies(test_values.plan_configuration)
pc_columns = pc.columns
pc_columns=np.array(pc_columns)
pc.columns=surface(pc_columns)
test_values[pc.columns]=pc
test_values=test_values.drop(columns = ['plan_configuration'])

In [417]:
x=pd.get_dummies(test_values.land_surface_condition)
x_columns = x.columns
x_columns=np.array(x_columns)
x.columns=surface(x_columns)
test_values[x.columns]=x
test_values=test_values.drop(columns = ['land_surface_condition'])

In [419]:
z=pd.get_dummies(test_values.foundation_type)
z_columns = z.columns
z_columns=np.array(z_columns)
z.columns=foundation(z_columns)
test_values[z.columns]=z
test_values=test_values.drop(columns = ['foundation_type'])

In [420]:
g=pd.get_dummies(test_values.geo_level_1_id)
g_columns = g.columns
g_columns=np.array(g_columns)
g.columns=foundation(g_columns)
test_values[g.columns]=g
test_values=test_values.drop(columns = ['geo_level_1_id'])

In [422]:
gf=pd.get_dummies(test_values.ground_floor_type)
gf_columns = gf.columns
gf_columns=np.array(gf_columns)
gf.columns=ground_floor_type(gf_columns)
test_values[gf.columns]=gf
test_values=test_values.drop(columns = ['ground_floor_type'])

In [423]:
test = lgb_clf.predict(test_values)

In [424]:
test_values_old  = pd.read_csv('../Archivos de datos/test_values.csv')

In [425]:
test_values_=pd.DataFrame()
test_values_['building_id']=test_values_old.building_id
test_values_['damage_grade']=pd.Series(test)
test_values_.to_csv('test_values_v4.csv',index=False)

In [426]:
test_values_.head(5)

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,2
3,745817,1
4,421793,3
